<a href="https://colab.research.google.com/github/VCMI-handson/Dataset/blob/master/Hands_on_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Segmentation

![대체 텍스트](https://drive.google.com/uc?export=view&id=1C_0gHRwgYOxWxZfcdzUTbJymACw801s2)

[출처](https://medium.com/zylapp/review-of-deep-learning-algorithms-for-object-detection-c1f3d437b852)


# Step 0. 실습환경 셋팅

##1. Google 계정 로그인

##2. 하단 링크 접속

##**[https://colab.research.google.com/github/VCMI-handson/210814](https://colab.research.google.com/github/VCMI-handson/210814)**


![대체 텍스트](https://drive.google.com/uc?export=view&id=1dzZvu44EMw7LQ7iGJfutXSuaZeK_6VoA)

![대체 텍스트](https://drive.google.com/uc?export=view&id=1puX_WuNVBWdsm9qJwulz2uTniUcDYaoE)

# Step 1. 데이터셋 불러들이기

##Lung segmentation from Chest X-Ray dataset

https://www.kaggle.com/nikhilpandey360/lung-segmentation-from-chest-x-ray-dataset

![대체 텍스트](https://drive.google.com/uc?export=view&id=1Oa-iXD9IMQre4Ao7CyslObAtZRBCskvB)
![대체 텍스트](https://drive.google.com/uc?export=view&id=1lz6EHChnPtCbkH6XI_Tm0_tEpJWnkb1a)
![대체 텍스트](https://drive.google.com/uc?export=view&id=1ZK09wg_gJGGCyKnvkSkgxcpl1Pz4esDa)


###Lung_Segmentation.zip
###256x256x3
###566 datasets [image, label]

In [ ]:
!rm -rf *
!wget https://github.com/mi2rl/datasets/raw/master/Lung_Segmentation.zip

In [ ]:
!ls

In [ ]:
!unzip Lung_Segmentation.zip

**(DICOM 을 PNG로 바꾸고 싶은 경우)**

In [ ]:
!pwd

In [ ]:
!pip install pydicom
!pip install pillow
!pip install numpy

In [ ]:
import pydicom as dicom
import numpy as np
from PIL import Image

dcm_path = "./Samples/dcm"
data_path = "./Samples/image"
os.mkdir(data_path)
files = os.listdir(dcm_path)

for f in files:
    image_path = os.path.join(dcm_path, f)

    f1 = os.path.splitext(f)[0]
    d1 = os.path.join('{}.png'.format(f1))

    png_path = os.path.join(data_path, d1)

    ds = dicom.dcmread(image_path)

    new_image = ds.pixel_array.astype(float)
    scaled_image = (np.maximum(new_image, 0) / new_image.max()) * 255.0
    scaled_image = np.uint8(scaled_image)
    final_image = Image.fromarray(scaled_image)
    
    final_image.save(png_path)

**Lung Segmentation 데이터 설정하기**

In [ ]:
import numpy as np
import os

from skimage.io import imread
from skimage.transform import resize
from sklearn.model_selection import train_test_split

IMG_WIDTH = 256
IMG_HEIGHT = 256
IMG_CHANNELS = 3

data_path = "./Lung_Segmentation"

files = os.listdir(os.path.join(data_path, 'image'))
file_headers = []  #python list
for f in files:
    f1 = os.path.splitext(f)[0]
    file_headers.append(f1)
#print(file_headers)

X_all = np.zeros((len(file_headers), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
y_all = np.zeros((len(file_headers), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.float32)

count = 0
for fh in file_headers:
    f1 = os.path.join(data_path, 'image', '{}.png'.format(fh))
    l1 = os.path.join(data_path, 'label', '{}.png'.format(fh))
    
    img = imread(f1)[:,:,:IMG_CHANNELS]
    mask = imread(l1)
    mask = np.expand_dims(mask, axis=-1)

    X_all[count] = img
    y_all[count] = mask
    
    count += 1

**딥러닝을 위한 데이터 전처리**

In [ ]:
X_all = X_all.astype('float32') / 255.
y_all = y_all.astype('float32') / 255.

**학습, 검증, 테스트 데이터 셋으로 분리**

In [ ]:
# fix random seed for reproductibility
seed = 7
np.random.seed(seed)

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2)

print('X_train',X_train.shape)
print('X_valid',X_valid.shape)
print('X_test',X_test.shape)
print('y_train',y_train.shape)
print('y_valid',y_valid.shape)
print('y_test',y_test.shape)

# Step 2. 데이터 살펴보기

In [ ]:
import matplotlib.pyplot as plt

def plotTrainData(a,b,c):
    for i in range(3):
        ix = np.random.randint(0, len(a))
        plt.subplot(1,2,1)
        plt.title("X_" + c)
        plt.imshow(a[ix])
        plt.axis('off')
        plt.subplot(1,2,2)
        plt.title("y_" + c)
        plt.imshow(np.squeeze(b[ix]), 'gray')
        plt.axis('off')
        plt.show()
        
plotTrainData(X_train,y_train, 'train')
plotTrainData(X_valid,y_valid, 'valid')
plotTrainData(X_test,y_test, 'test')

# Step 3. 첫번째 영상분할 모델 (U-Net)

![대체 텍스트](https://drive.google.com/uc?export=view&id=1vSx_i5kIBjaHTk5pvaa-yOXTX_69UzZh)

In [ ]:
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras import backend as keras
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, LearningRateScheduler

def unet(input_size=(256,256,3)):
    inputs = Input(input_size)
    
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = BatchNormalization()(conv4)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    conv4 = BatchNormalization()(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = BatchNormalization()(conv5)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)
    conv5 = BatchNormalization()(conv5)
    
    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    conv6 = BatchNormalization()(conv6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)
    conv6 = BatchNormalization()(conv6)
    
    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    conv7 = BatchNormalization()(conv7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)
    conv7 = BatchNormalization()(conv7)
    
    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
    conv8 = BatchNormalization()(conv8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)
    conv8 = BatchNormalization()(conv8)
    
    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    conv9 = BatchNormalization()(conv9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)
    conv9 = BatchNormalization()(conv9)

    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

    return Model(inputs=[inputs], outputs=[conv10])

**loss 계산**

In [ ]:
from keras import backend as K

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + 1) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1)

def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

**생성한 모델 수행**

In [ ]:
# build the model
model = unet()
sgd = tf.keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss=dice_coef_loss, optimizer=sgd, metrics=['accuracy'])

# fix random seed for reproductibility
seed = 7
np.random.seed(seed)

# FIT THE MODEL - OPTIMIZATION
hist = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=20, batch_size=8, verbose=1)
model.save('unet.h5')

# 학습과정 살펴보기
fig, loss_ax = plt.subplots()

acc_ax = loss_ax.twinx()

loss_ax.plot(hist.history['loss'], 'y', label='train loss')
loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')
loss_ax.set_ylim([-1.0, -0.0])

acc_ax.plot(hist.history['accuracy'], 'b', label='train acc')
acc_ax.plot(hist.history['val_accuracy'], 'g', label='val acc')
acc_ax.set_ylim([0.0, 1.0])

loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('accuray')

loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')

plt.show()

**결과 확인**

In [ ]:
import matplotlib.pyplot as plt

def plotPredictions(X_train_, y_train_, X_valid_, y_valid_, X_test_, y_test_, simpleFCN):
    model = simpleFCN     

    ix = np.random.randint(0, len(X_train_))
    input_ = X_train_[ix:ix+1]
    mask_ = y_train_[ix:ix+1]
    preds_train = model.predict(input_)
    preds_train_t = (preds_train > 0.5).astype(np.uint8)
    plt.figure(figsize=(10,10))
    plt.subplot(1,3,1)
    plt.title("X_train")
    plt.axis('off')
    plt.imshow(input_[0])
    plt.subplot(1,3,2)
    plt.title("Y_train")
    plt.axis('off')
    plt.imshow(mask_[0][:,:,0], 'gray')
    plt.subplot(1,3,3)
    ret = model.evaluate(input_, mask_)
    plt.title("Prediction: %.4f" % (ret[1]))
    plt.axis('off')
    plt.imshow(preds_train_t[0][:,:,0], 'gray')
    plt.show()
    
    ix = np.random.randint(0, len(X_valid_))
    input_ = X_valid_[ix:ix+1]
    mask_ = y_valid_[ix:ix+1]
    preds_valid = model.predict(input_)
    preds_valid_t = (preds_valid > 0.5).astype(np.uint8)
    plt.figure(figsize=(10,10))
    plt.subplot(1,3,1)
    plt.title("X_valid")
    plt.axis('off')
    plt.imshow(input_[0])
    plt.subplot(1,3,2)
    plt.title("Y_valid")
    plt.axis('off')
    plt.imshow(mask_[0][:,:,0], 'gray')
    plt.subplot(1,3,3)
    ret = model.evaluate(input_, mask_)
    plt.title("Prediction: %.4f" % (ret[1]))
    plt.axis('off')
    plt.imshow(preds_valid_t[0][:,:,0], 'gray')
    plt.show()
    
    ix = np.random.randint(0, len(X_test_))
    input_ = X_test_[ix:ix+1]
    mask_ = y_test_[ix:ix+1]
    preds_test = model.predict(input_)
    preds_test_t = (preds_test > 0.5).astype(np.uint8)
    plt.figure(figsize=(10,10))
    plt.subplot(1,3,1)
    plt.title("X_test")
    plt.axis('off')
    plt.imshow(input_[0])
    plt.subplot(1,3,2)
    plt.title("Y_test")
    plt.axis('off')
    plt.imshow(mask_[0][:,:,0], 'gray')
    plt.subplot(1,3,3)
    ret = model.evaluate(input_, mask_)
    plt.title("Prediction: %.4f" % (ret[1]))
    plt.axis('off')
    plt.imshow(preds_test_t[0][:,:,0], 'gray')
    plt.show()

In [ ]:
plotPredictions(X_train, y_train, X_valid, y_valid, X_test, y_test, model)

# Step 4. 2번째 영상분할 모델 (Attention U-Net)

![대체 텍스트](https://github.com/lixiaolei1982/Keras-Implementation-of-U-Net-R2U-Net-Attention-U-Net-Attention-R2U-Net.-/blob/master/img/AttU-Net.png?raw=true)

In [ ]:
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras import backend as keras
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, LearningRateScheduler

def attention(x, g, n_filters, padding='same', activation='relu'):
  original_x = x
  x_net = original_x
  g_net = g

  x_conv = Conv2D(n_filters, (1, 1), activation=None, padding=padding)(x_net)  # contracting_path_results
  x_conv = BatchNormalization()(x_conv)
  g_conv = Conv2D(n_filters, (1, 1), activation=None, padding=padding)(g_net)  # up_results
  g_conv = BatchNormalization()(g_conv)
  
  net = add([x_conv, g_conv])
  net = Activation('relu')(net)
  net = Conv2D(1, (1, 1), activation=None, padding=padding)(net)
  net = BatchNormalization()(net)
  net_a = Activation('sigmoid')(net)
  net_result = multiply([original_x, net_a])

  return net_result

In [ ]:
def Attunet(input_size=(256,256,3)):
    inputs = Input(input_size)
    
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = BatchNormalization()(conv4)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    conv4 = BatchNormalization()(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = BatchNormalization()(conv5)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)
    conv5 = BatchNormalization()(conv5)
    
    up6 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5)
    att6 = attention(conv4, up6, 256)
    concat6 = concatenate([up6, att6], axis=3)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(concat6)
    conv6 = BatchNormalization()(conv6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)
    conv6 = BatchNormalization()(conv6)
    
    up7 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6)
    att7 = attention(conv3, up7, 128)
    concat7 = concatenate([up7, att7], axis=3)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(concat7)
    conv7 = BatchNormalization()(conv7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)
    conv7 = BatchNormalization()(conv7)
    
    up8 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7)
    att8 = attention(conv2, up8, 64)
    concat8 = concatenate([up8, att8], axis=3)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(concat8)
    conv8 = BatchNormalization()(conv8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)
    conv8 = BatchNormalization()(conv8)
    
    up9 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8)
    att9 = attention(conv1, up9, 32)
    concat9 = concatenate([up9, att9], axis=3)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(concat9)
    conv9 = BatchNormalization()(conv9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)
    conv9 = BatchNormalization()(conv9)

    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

    return Model(inputs=[inputs], outputs=[conv10])

**생성한 모델 수행**

In [ ]:
# build the model
model = Attunet()
sgd = tf.keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss=dice_coef_loss, optimizer=sgd, metrics=['accuracy'])

# fix random seed for reproductibility
seed = 7
np.random.seed(seed)

# FIT THE MODEL - OPTIMIZATION
hist = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=20, batch_size=8, verbose=1)
model.save('Attunet.h5')

# 학습과정 살펴보기
fig, loss_ax = plt.subplots()

acc_ax = loss_ax.twinx()

loss_ax.plot(hist.history['loss'], 'y', label='train loss')
loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')
loss_ax.set_ylim([-1.0, -0.0])

acc_ax.plot(hist.history['accuracy'], 'b', label='train acc')
acc_ax.plot(hist.history['val_accuracy'], 'g', label='val acc')
acc_ax.set_ylim([0.0, 1.0])

loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('accuray')

loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')

plt.show()

**결과 확인**

In [ ]:
plotPredictions(X_train, y_train, X_valid, y_valid, X_test, y_test, model)

# Step 5. 3번째 영상분할 모델 (Recurrent Residual U-Net: R2U-Net)

![대체 텍스트](https://github.com/lixiaolei1982/Keras-Implementation-of-U-Net-R2U-Net-Attention-U-Net-Attention-R2U-Net.-/blob/master/img/R2U-Net.png?raw=true)

![대체 텍스트](https://drive.google.com/uc?export=view&id=1T0nksBa4Pe64eurM_fTbB26B1H4T9XpO)


In [ ]:
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras import backend as keras
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, LearningRateScheduler

def recurrent_block(input_layer, n_filters, padding='same', activation='relu'):
    
    input_conv = Conv2D(n_filters, (1, 1), activation=None, padding='same')(input_layer)

    conv1 = Conv2D(n_filters, (3, 3), padding='same')(input_layer) 
    conv1 = BatchNormalization()(conv1)
    conv1 = Activation('relu')(conv1)

    conv2 = Conv2D(n_filters, (3, 3), padding='same')(add([input_conv, conv1])) 
    conv2 = BatchNormalization()(conv2)
    conv2 = Activation('relu')(conv2)
    
    return conv2


def r2_block(input_layer, n_filters):
    
    r_block = recurrent_block(input_layer, n_filters)
    r_block = recurrent_block(r_block, n_filters)

    input_conv = Conv2D(n_filters, (1, 1), activation=None, padding='same')(input_layer)
    
    out_layer = add([r_block, input_conv])

    return out_layer

In [ ]:
def r2unet(input_size=(256,256,3)):
    inputs = Input(input_size)
    
    rrcnn1 = r2_block(inputs, 32)
    pool1 = MaxPooling2D(pool_size=(2, 2))(rrcnn1)

    rrcnn2 = r2_block(pool1, 64)
    pool2 = MaxPooling2D(pool_size=(2, 2))(rrcnn2)

    rrcnn3 = r2_block(pool2, 128)
    pool3 = MaxPooling2D(pool_size=(2, 2))(rrcnn3)

    rrcnn4 = r2_block(pool3, 256)
    pool4 = MaxPooling2D(pool_size=(2, 2))(rrcnn4)

    rrcnn5 = r2_block(pool4, 512)

    up6 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(rrcnn5)
    concat6 = concatenate([up6, rrcnn4], axis = 3)
    rrcnn6 = r2_block(concat6, 256)

    up7 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(rrcnn6)
    concat7 = concatenate([up7, rrcnn3], axis = 3)
    rrcnn7 = r2_block(concat7, 128)

    up8 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(rrcnn7)
    concat8 = concatenate([up8, rrcnn2], axis = 3)
    rrcnn8 = r2_block(concat8, 64)

    up9 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(rrcnn8)
    concat9 = concatenate([up9, rrcnn1], axis = 3)
    rrcnn9 = r2_block(concat9, 32)

    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(rrcnn9)

    return Model(inputs=[inputs], outputs=[conv10])

**생성한 모델 수행**

In [ ]:
# build the model
model = r2unet()
sgd = tf.keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss=dice_coef_loss, optimizer=sgd, metrics=['accuracy'])

# fix random seed for reproductibility
seed = 7
np.random.seed(seed)

# FIT THE MODEL - OPTIMIZATION
hist = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=20, batch_size=8, verbose=1)
model.save('r2unet.h5')

# 학습과정 살펴보기
fig, loss_ax = plt.subplots()

acc_ax = loss_ax.twinx()

loss_ax.plot(hist.history['loss'], 'y', label='train loss')
loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')
loss_ax.set_ylim([-1.0, -0.0])

acc_ax.plot(hist.history['accuracy'], 'b', label='train acc')
acc_ax.plot(hist.history['val_accuracy'], 'g', label='val acc')
acc_ax.set_ylim([0.0, 1.0])

loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('accuray')

loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')

plt.show()

**결과 확인**

In [ ]:
plotPredictions(X_train, y_train, X_valid, y_valid, X_test, y_test, model)